### 1. IMPORTACIÓN DE LIBRERÍAS

In [52]:
import pandas as pd
import numpy as np
import os

### 2. CARGA DE LOS DATOS (DE MOMENTO CONSIDERANDO UN .CSV)

In [53]:
def load_data(file_path: str) -> pd.DataFrame | None:
    """
    Carga un conjunto de datos desde una ruta de archivo CSV.

    Esta función encapsula la lógica de lectura de datos con Pandas,
    incluyendo un manejo básico de errores si el archivo no se encuentra.

    Args:
        file_path (str): La ruta al archivo .csv que se va a cargar.

    Returns:
        pd.DataFrame | None: Un DataFrame de Pandas con los datos cargados,
        o None si ocurre un error (ej. archivo no encontrado).
    """
    try:
        # Intenta leer el archivo CSV y lo carga en un DataFrame
        df = pd.read_csv(file_path)
        print(f"Datos cargados exitosamente desde: {file_path}")
        return df
    except FileNotFoundError:
        # Manejo de error si el archivo no existe en la ruta especificada
        print(f"Error: El archivo no fue encontrado en la ruta: {file_path}")
        return None
    except Exception as e:
        # Manejo de otros posibles errores durante la carga
        print(f"Ocurrió un error inesperado al cargar el archivo: {e}")
        return None

### 3. DIAGNÓSTICO INICIAL

In [54]:
def get_data_overview(df: pd.DataFrame, df_name: str = "DataFrame") -> None:
    """
    Imprime un resumen completo y diagnóstico de un DataFrame.

    Incluye dimensiones, tipos de datos, estadísticas descriptivas,
    conteo de duplicados y porcentaje de valores nulos.

    Args:
        df (pd.DataFrame): El DataFrame que se va a analizar.
        df_name (str): Un nombre opcional para el DataFrame que se mostrará en los reportes.
    """
    print(f"\n===============ANÁLISIS EXPLORATORIO RÁPIDO PARA: '{df_name}'===============")

    # 1. Dimensiones del DataFrame
    print(f"\n**Dimensiones:** {df.shape[0]} filas y {df.shape[1]} columnas.")

    # 2. Tipos de datos y valores no nulos
    print("\n**Tipos de Datos y Valores No Nulos:**")
    df.info()

    # 3. Estadísticas Descriptivas para variables numéricas
    print("\n**Estadísticas Descriptivas (Numéricas):**")
    # Usamos .T para transponer la tabla y hacerla más legible
    print(df.describe().T)

    # 4. Conteo de filas duplicadas
    duplicates = df.duplicated().sum()
    print(f"\n**Filas Duplicadas:** {duplicates} filas duplicadas encontradas.")

    # 5. Porcentaje de Valores Nulos por columna
    null_percentage = (df.isnull().sum() / len(df)) * 100
    null_info = null_percentage[null_percentage > 0].sort_values(ascending=False)
    
    if not null_info.empty:
        print("\n**Porcentaje de Valores Nulos (>0%):**")
        print(null_info)
    else:
        print("\n**No se encontraron valores nulos.**")
    
    print("\nFin del análisis.")

### 4. LIMPIEZA DE LOS DATOS

##### Validación del esquema. Eliminación de columnas no identificadas

In [55]:
def drop_unnecessary_columns(df: pd.DataFrame, valid_columns: list) -> pd.DataFrame:
    """
    Elimina las columnas de un DataFrame que no están en una lista de columnas válidas.

    Esta función es útil para asegurar que el DataFrame solo contenga las columnas
    esperadas según el esquema definido.

    Args:
        df (pd.DataFrame): El DataFrame a limpiar.
        valid_columns (list): Una lista de strings con los nombres de las
                              columnas que deben permanecer.

    Returns:
        pd.DataFrame: Un nuevo DataFrame que solo contiene las columnas válidas.
    """

    # Identifica las columnas actuales del DataFrame
    current_columns = df.columns.tolist()
    
    # Encuentra las columnas que están en el DataFrame pero no en la lista de válidas
    cols_to_drop = [col for col in current_columns if col not in valid_columns]

    if cols_to_drop:
        print(f"\nColumnas a eliminar: {cols_to_drop}")
        # Elimina las columnas identificadas y devuelve una copia del df modificado
        df_cleaned = df.drop(columns=cols_to_drop)
        print("Columnas innecesarias eliminadas.")
    else:
        print("No se encontraron columnas innecesarias. El esquema es correcto.")
        df_cleaned = df.copy() # Devuelve una copia para mantener la consistencia

    return df_cleaned

##### Conversión del tipo correcto de columnas

In [56]:
def correct_initial_data_types(df: pd.DataFrame) -> pd.DataFrame:
    """
    Corrige los tipos de datos de columnas específicas a numérico y fecha.
    Las categóricas de momento igual se consideran como numéricas, para facilitar el manejo de inválidos.

    Usa 'errors=coerce' para convertir valores no válidos en NaN,
    facilitando su manejo posterior.

    Args:
        df (pd.DataFrame): El DataFrame a procesar.

    Returns:
        pd.DataFrame: Un nuevo DataFrame con los tipos de datos corregidos.
    """

    df_corrected = df.copy()
    print("\nIniciando corrección de tipos de datos...")

    # Define los grupos de columnas
    numeric_cols = ['temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered', 'cnt',
                    'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit']
    
    # 1. Procesa columnas numéricas
    for col in numeric_cols:
        # Omite si la columna no existe (por si fue eliminada antes)
        if col in df_corrected.columns:
            df_corrected[col] = pd.to_numeric(df_corrected[col], errors='coerce')

    # 2. Procesa columna de fecha
    if 'dteday' in df_corrected.columns:
        # Usamos format='mixed' para manejar explícitamente los formatos múltiples
        df_corrected['dteday'] = pd.to_datetime(
            df_corrected['dteday'], 
            errors='coerce', 
            format='mixed' # Esta es la solución
        )
        
    print("Tipos de datos corregidos de forma semántica.")
    return df_corrected

##### Manejo de valores no válidos

In [57]:
def handle_invalid_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Valida los datos contra un conjunto de reglas y convierte los inválidos a NaN.

    Args:
        df (pd.DataFrame): El DataFrame con tipos de datos ya corregidos.

    Returns:
        pd.DataFrame: Un nuevo DataFrame con los valores inválidos convertidos a NaN.
    """

    df_validated = df.copy()
    print("\nIniciando validación de valores...")

    # Diccionario de reglas de validación
    # Para categóricas: lista de valores permitidos
    # Para numéricas: tupla con (valor_mínimo, valor_máximo)
    validation_rules = {
        'dteday': (pd.to_datetime('2011-01-01'), pd.to_datetime('2012-12-31')),
        'season': [1, 2, 3, 4],
        'yr': [0, 1],
        'mnth': list(range(1, 13)),
        'hr': list(range(0, 24)),
        'holiday': [0, 1],
        'weekday': list(range(0, 7)),
        'workingday': [0, 1],
        'weathersit': [1, 2, 3, 4],
        'hum': (0.0, 1.0),
        'windspeed': (0.0, 1.0),
        'cnt': (0, float('inf')) # El conteo no puede ser negativo
    }

    for column, rule in validation_rules.items():
        if column in df_validated.columns:
            # Conteo inicial de nulos para reporte
            initial_nulls = df_validated[column].isnull().sum()

            # Validación para variables categóricas (regla es una lista)
            if isinstance(rule, list):
                invalid_mask = ~df_validated[column].isin(rule)
            # Validación para variables numéricas (regla es una tupla)
            elif isinstance(rule, tuple):
                min_val, max_val = rule
                invalid_mask = (df_validated[column] < min_val) | (df_validated[column] > max_val)
            
            # Reemplaza los valores que no cumplen la regla con NaN
            df_validated.loc[invalid_mask, column] = np.nan
            
            # Reporta cuántos valores inválidos se encontraron y corrigieron
            final_nulls = df_validated[column].isnull().sum()
            newly_invalid = final_nulls - initial_nulls
            if newly_invalid > 0:
                print(f"  -> Columna '{column}': {newly_invalid} valores inválidos convertidos a NaN.")

    print("Validación de valores completada.")
    return df_validated

##### Imputación de valores faltantes

In [58]:
def date_to_season(date_obj: pd.Timestamp) -> int:
    """
    Convierte una fecha completa a la estación correspondiente de forma precisa,
    considerando los días de corte (solsticios y equinoccios).
    Dataset: 1:invierno, 2:primavera, 3:verano, 4:otoño.
    """
    if pd.isna(date_obj):
        return np.nan
        
    month = date_obj.month
    day = date_obj.day

    # Invierno: Desde 21 de Dic hasta 20 de Mar
    if (month == 12 and day >= 21) or (month in [1, 2]) or (month == 3 and day < 21):
        return 1
    # Primavera: Desde 21 de Mar hasta 20 de Jun
    elif (month == 3 and day >= 21) or (month in [4, 5]) or (month == 6 and day < 21):
        return 2
    # Verano: Desde 21 de Jun hasta 22 de Sep
    elif (month == 6 and day >= 21) or (month in [7, 8]) or (month == 9 and day < 23):
        return 3
    # Otoño: Desde 23 de Sep hasta 20 de Dic
    else:
        return 4

In [59]:
def handle_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Gestiona valores nulos: primero elimina filas irrecuperables y luego
    imputa los nulos restantes de forma contextual.
    """

    df_processed = df.copy()
    print("\nIniciando manejo de valores nulos...")
    
    initial_rows = len(df_processed)
    
    # --- ESTRATEGIA 1: ELIMINACIÓN DE FILAS CRÍTICAS ---
    print("\n  -> Paso 1: Eliminando filas con datos críticos faltantes...")
    critical_cols = ['dteday', 'hr', 'holiday', 'workingday', 'casual', 'registered', 'cnt']
    df_processed.dropna(subset=critical_cols, inplace=True)
    rows_deleted = initial_rows - len(df_processed)
    if rows_deleted > 0:
        print(f"Se eliminaron {rows_deleted} filas.")

    # --- ESTRATEGIA 2: IMPUTACIÓN CONTEXTUAL ---
    print("\n  -> Paso 2: Imputando valores restantes de forma contextual...")
    
    # Derivación por fecha (solo para celdas vacías)
    mask_yr = df_processed['yr'].isna()
    df_processed.loc[mask_yr, 'yr'] = df_processed.loc[mask_yr, 'dteday'].dt.year - 2011
    
    mask_mnth = df_processed['mnth'].isna()
    df_processed.loc[mask_mnth, 'mnth'] = df_processed.loc[mask_mnth, 'dteday'].dt.month
    
    mask_weekday = df_processed['weekday'].isna()
    df_processed.loc[mask_weekday, 'weekday'] = (df_processed.loc[mask_weekday, 'dteday'].dt.weekday + 1) % 7
    
    mask_season = df_processed['season'].isna()
    df_processed.loc[mask_season, 'season'] = df_processed.loc[mask_season, 'dteday'].apply(date_to_season)

    # Imputación estadística para el clima (sin cambios)
    weather_cols = ['weathersit', 'temp', 'atemp', 'hum', 'windspeed']
    for column in weather_cols:
        if column in df_processed.columns and df_processed[column].isnull().any():
            median_val = df_processed[column].median()
            df_processed[column] = df_processed[column].fillna(median_val)

    print("Proceso de imputación de nulos finalizado.")
    return df_processed

##### Revisión de registros con inconsistencias

In [60]:
def check_inconsistencies(df: pd.DataFrame) -> pd.DataFrame:
    """
    Verifica y corrige inconsistencias lógicas en el DataFrame.

    - Regla 1: Valida 'season', 'yr', 'mnth', 'weekday' contra 'dteday'.
    - Regla 2: Valida 'workingday' contra 'weekday' y 'holiday'.
    - Regla 3: Valida 'cnt' = 'casual' + 'registered' y elimina filas inconsistentes.
    """

    df_consistent = df.copy()
    print("\nVerificando inconsistencias lógicas...")

    # --- Regla 1: Consistencia de variables de tiempo vs. 'dteday' ---
    print("\n  -> Validando consistencia de fecha (yr, mnth, season, weekday)...")
    
    # Se calculan los valores correctos a partir de la fecha
    correct_yr = df_consistent['dteday'].dt.year - 2011
    correct_mnth = df_consistent['dteday'].dt.month
    correct_weekday = (df_consistent['dteday'].dt.weekday + 1) % 7
    correct_season = df_consistent['dteday'].apply(date_to_season)

    # Se comparan y corrigen
    yr_inconsistencies = (df_consistent['yr'] != correct_yr).sum()
    mnth_inconsistencies = (df_consistent['mnth'] != correct_mnth).sum()
    weekday_inconsistencies = (df_consistent['weekday'] != correct_weekday).sum()
    season_inconsistencies = (df_consistent['season'] != correct_season).sum()
    
    df_consistent['yr'] = correct_yr
    df_consistent['mnth'] = correct_mnth
    df_consistent['weekday'] = correct_weekday
    df_consistent['season'] = correct_season
    
    print(f"    - Corregidas {yr_inconsistencies} inconsistencias en 'yr'.")
    print(f"    - Corregidas {mnth_inconsistencies} inconsistencias en 'mnth'.")
    print(f"    - Corregidas {weekday_inconsistencies} inconsistencias en 'weekday'.")
    print(f"    - Corregidas {season_inconsistencies} inconsistencias en 'season'.")
    
    # --- Regla 2: Consistencia de 'workingday' ---
    print("\n  -> Validando consistencia de 'workingday'...")
    
    # Se calcula el valor correcto: no es fin de semana (0 o 6) Y no es festivo (0)
    correct_workingday = ((df_consistent['weekday'].isin([0, 6])) | (df_consistent['holiday'] == 1)).apply(lambda x: 0 if x else 1)
    
    workingday_inconsistencies = (df_consistent['workingday'] != correct_workingday).sum()
    df_consistent['workingday'] = correct_workingday
    print(f"    - Corregidas {workingday_inconsistencies} inconsistencias en 'workingday'.")

    # --- Regla 3: Consistencia de los conteos ('cnt') ---
    print("\n  -> Validando consistencia de 'cnt' vs 'casual' + 'registered'...")
    
    # Se identifican las filas donde la suma no cuadra
    inconsistent_sum_mask = df_consistent['cnt'] != (df_consistent['casual'] + df_consistent['registered'])
    
    rows_to_drop = inconsistent_sum_mask.sum()
    
    if rows_to_drop > 0:
        df_consistent = df_consistent[~inconsistent_sum_mask]
        print(f"    - Se eliminaron {rows_to_drop} filas por inconsistencia en la suma de conteos.")
    else:
        print("    - No se encontraron inconsistencias en la suma de conteos.")

    print("\nVerificación de inconsistencias completada.")
    return df_consistent

##### Eliminación de registros duplicados

In [61]:
def drop_duplicate_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    Encuentra y elimina filas completamente duplicadas en el DataFrame.
    """
    print("\nVerificando filas duplicadas...")
    
    initial_rows = len(df)
    
    # Se eliminan las filas duplicadas
    df_unique = df.drop_duplicates()
    
    final_rows = len(df_unique)
    rows_dropped = initial_rows - final_rows
    
    if rows_dropped > 0:
        print(f"Se eliminaron {rows_dropped} filas duplicadas.")
    else:
        print("No se encontraron filas duplicadas.")
        
    return df_unique

##### Conversión final de columnas categóricas y numéricas enteras

In [62]:
def finalize_data_types(df: pd.DataFrame) -> pd.DataFrame:
    """
    Paso Final: Convierte las columnas a sus tipos semánticos finales (category, int).
    Se ejecuta después de que toda la limpieza e imputación han sido completadas.
    """
    df_finalized = df.copy()
    print("\nPuliendo los tipos de datos finales...")

    # Define qué columnas deben ser categóricas y cuáles de conteo (enteros)
    categorical_cols = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit']
    count_cols = ['casual', 'registered', 'cnt']

    # Convierte las columnas categóricas
    for col in categorical_cols:
        if col in df_finalized.columns:
            df_finalized[col] = df_finalized[col].astype('category')
    
    # Convierte las columnas de conteo a entero
    for col in count_cols:
         if col in df_finalized.columns:
            df_finalized[col] = df_finalized[col].astype(int)

    # Reinicia el index
    df_finalized = df_finalized.reset_index(drop=True)

    print("Tipos de datos finalizados.")
    return df_finalized

### 5. EJECUCIÓN DE PIPELINE COMPLETO DE LIMPIEZA

In [63]:
def run_cleaning_pipeline(raw_data_path: str, expected_columns: list) -> pd.DataFrame | None:
    """
    Ejecuta el pipeline completo de limpieza de datos sobre un archivo CSV.

    Args:
        raw_data_path (str): La ruta al archivo CSV crudo.
        expected_columns (list): La lista de columnas esperadas en el dataset.

    Returns:
        pd.DataFrame | None: Un DataFrame limpio y procesado, o None si la carga inicial falla.
    """
    print("INICIANDO PIPELINE DE LIMPIEZA DE DATOS")
    
    # --- PASO 1: CARGA ---
    print("\n--- PASO 1: CARGA DEL DATASET CRUDO ---")
    df = load_data(raw_data_path)
    
    if df is None:
        return None # Retorna None si la carga falla

    # --- PASO 2: APLICANDO TRANSFORMACIONES ---
    # Se encadenan todas las funciones de limpieza de forma secuencial.
    print("\n--- PASO 2: APLICANDO TRANSFORMACIONES DE LIMPIEZA ---")
    df = drop_unnecessary_columns(df, expected_columns)
    df = correct_initial_data_types(df)
    df = handle_invalid_values(df)
    df = handle_missing_values(df)
    df = drop_duplicate_rows(df)
    df = check_inconsistencies(df)
    df = finalize_data_types(df) # El resultado de esta es el df final
    
    print("\nPipeline de limpieza completado exitosamente.")
    return df

### 6. EXPORTACIÓN DE DATAFRAME A .CSV

In [64]:
def save_dataframe_to_csv(df: pd.DataFrame, file_path: str) -> None:
    """
    Guarda un DataFrame de Pandas en un archivo CSV.

    Verifica si el directorio de destino existe y lo crea si es necesario.
    Guarda el archivo sin el índice de Pandas.

    Args:
        df (pd.DataFrame): El DataFrame que se va a guardar.
        file_path (str): La ruta completa del archivo, incluyendo el nombre y la extensión .csv.
    """
    try:
        # Extrae el directorio de la ruta del archivo
        directory = os.path.dirname(file_path)
        
        # Crea el directorio si no existe
        if not os.path.exists(directory):
            os.makedirs(directory)
            print(f"\nDirectorio creado: {directory}")
            
        # Guarda el DataFrame en CSV sin el índice
        df.to_csv(file_path, index=False)
        print(f"\nDataFrame guardado exitosamente en: {file_path}")
        
    except Exception as e:
        print(f"\nError al guardar el archivo: {e}")

### FUNCIÓN MAIN

In [65]:
def main():
    """
    Función principal que configura y ejecuta el pipeline de limpieza,
    y luego presenta un reporte del resultado.
    """
    # --- 0. CONFIGURACIÓN ---
    RAW_DATA_PATH = '../data/raw/bike_sharing_modified.csv'
    EXPECTED_COLUMNS = [
        'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 
        'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 
        'casual', 'registered', 'cnt'
    ]
    
    # --- 1. EJECUCIÓN DEL PIPELINE ---
    cleaned_df = run_cleaning_pipeline(
        raw_data_path=RAW_DATA_PATH,
        expected_columns=EXPECTED_COLUMNS
    )
    
    # --- 2. REPORTE FINAL ---
    if cleaned_df is not None:
        print("\n==========================================================")
        print("REPORTE FINAL DEL DATAFRAME LIMPIO")
        print("==========================================================")
        
        print("\nInformación General y Tipos de Datos Finales:")
        cleaned_df.info()
        
        print("\nResumen Estadístico Final:")
        print(cleaned_df.describe(include='all').T)
        
        print("\nPrimeras 5 Filas del Dataset Limpio:")
        print(cleaned_df.head())

        # --- GUARDADO DEL RESULTADO ---
        CLEANED_DATA_PATH = '../data/interim/bike_sharing_cleaned.csv'
        
        # Se llama a la función para guardar el archivo
        save_dataframe_to_csv(df=cleaned_df, file_path=CLEANED_DATA_PATH)
    else:
        print("El pipeline no generó un resultado debido a un error en la carga de datos.")

# =============================================================================
# Punto de Entrada del Script
# =============================================================================
if __name__ == '__main__':
    main()

INICIANDO PIPELINE DE LIMPIEZA DE DATOS

--- PASO 1: CARGA DEL DATASET CRUDO ---
Datos cargados exitosamente desde: ../data/raw/bike_sharing_modified.csv

--- PASO 2: APLICANDO TRANSFORMACIONES DE LIMPIEZA ---

Columnas a eliminar: ['instant', 'mixed_type_col']
Columnas innecesarias eliminadas.

Iniciando corrección de tipos de datos...
Tipos de datos corregidos de forma semántica.

Iniciando validación de valores...
  -> Columna 'season': 168 valores inválidos convertidos a NaN.
  -> Columna 'yr': 136 valores inválidos convertidos a NaN.
  -> Columna 'mnth': 177 valores inválidos convertidos a NaN.
  -> Columna 'hr': 174 valores inválidos convertidos a NaN.
  -> Columna 'holiday': 96 valores inválidos convertidos a NaN.
  -> Columna 'weekday': 158 valores inválidos convertidos a NaN.
  -> Columna 'workingday': 120 valores inválidos convertidos a NaN.
  -> Columna 'weathersit': 180 valores inválidos convertidos a NaN.
  -> Columna 'hum': 167 valores inválidos convertidos a NaN.
  -> Co